In [0]:
%run ../.././start_up

In [0]:
logger = create_logger(notebook_name="landing_to_bronze", log_level="DEBUG")
logger.info("🚀 Initializing landing_to_bronze notebook")

# Extract frequently used config values into variables
catalog = pipeline_config["catalog"]
bronze_schema = pipeline_config["schemas"]["bronze"]
bronze_path = pipeline_config["paths"]["bronze_path"]
landing_schema= pipeline_config["schemas"]["landing"]
landing_path = pipeline_config["paths"]["landing_path"]
bronze_volume_path = pipeline_config["paths"]["bronze_volume_path"]
logs_schema = pipeline_config["schemas"]["logs"]
logger.info("Extracted frequently used config values into variables")

In [0]:

# === Create Volume ===
spark.sql(f"""
    CREATE VOLUME IF NOT EXISTS {catalog}.{bronze_schema}.{bronze_volume_name}
""")

# === Loop through each file and ingest ===
for file in tables_to_process:
    input_path = f"{landing_path}/{file}.csv"
    
    df = (
        spark.read.format("csv")
        .option("header", "true")
        .load(input_path)
        .withColumn("ingestion_date", current_timestamp())
    )

    # display(df)

    # Write to Delta inside the volume (as a folder for each table)
    df.coalesce(1).write.format("csv") \
        .option("header","true") \
        .mode("overwrite") \
        .save(f"{bronze_volume_path}/{file}")

    logger.info(f"✅ Ingested {file}.csv into volume {bronze_volume_name} under folder {file}")
